# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [11]:
import pandas as pd
import math
import numpy as np
import os

In [12]:
import get_pums as get
import pums as calc

In [13]:
#Search parameters
y1 = '2021'
#y0 = '2013'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

#north_pumas = ['0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,SMOCP'

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
1000000 = 1.000000

FHINCP - income flag - 1: yes | https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

SMOCP - selected owner costs (monthly) (not '00000': none / '-1')
https://api.census.gov/data/2021/acs/acs1/pums/variables/SMOCP.json

In [14]:
#AMI bands in 2021 by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [15]:
#Housing costs affordable to different AMI bands based on household size
aff_1p = {'30pct':461,'50pct':767,'80pct':1217,'100pct':1535}
aff_2p = {'30pct':527,'50pct':877,'80pct':1392,'100pct':1754}
aff_3p = {'30pct':609,'50pct':985,'80pct':1565,'100pct':1973}
aff_4p = {'30pct':735,'50pct':1096,'80pct':1738,'100pct':2192}
aff_5p = {'30pct':861,'50pct':1185,'80pct':1878,'100pct':2370}
aff_6p = {'30pct':987,'50pct':1272,'80pct':2017,'100pct':2545}
aff_7p = {'30pct':1113,'50pct':1360,'80pct':2156,'100pct':2720}
aff_8p = {'30pct':1239,'50pct':1447,'80pct':2295,'100pct':2894}

unit_afford = {'1':[0,461,767,1217,1535,1000000000],\
              '2':[0,527,877,1392,1535,1000000000],\
              '3':[0,609,985,1565,1973,1000000000],\
              '4':[0,735,1096,1738,2192,1000000000],\
              '5':[0,861,1185,1878,2370,1000000000],\
              '6':[0,987,1272,2017,2545,1000000000],\
              '7':[0,1113,1360,2156,2720,1000000000],\
              '8':[0,1239,1447,2295,2894,1000000000]}

inc_lbls = ['u30_ami','30_50_ami','50_80_ami','80_100_ami','o100_ami']

In [16]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [17]:
df = get.get_puma(sample,y1,data_cols)

In [18]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [19]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)
df['HINCP'] = df.ADJINC * df.HINCP

In [23]:
dff = df[(df.SMOCP!=0)&(df.SMOCP!=-1)].copy()

In [24]:
dff.head()

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,SMOCP,WGTP1,WGTP2,WGTP3,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,HHSz
3602,0400112,76214.6720,2.0,55.0,1.029928,1000000.0,3176.0,56.0,96.0,91.0,...,16.0,96.0,58.0,54.0,16.0,93.0,51.0,56.0,94.0,2
3607,0400116,138010.3520,6.0,53.0,1.029928,1000000.0,1628.0,15.0,16.0,62.0,...,58.0,16.0,17.0,60.0,57.0,18.0,90.0,58.0,54.0,6
3618,0400122,32339.7392,2.0,109.0,1.029928,1000000.0,717.0,96.0,152.0,114.0,...,98.0,100.0,108.0,128.0,108.0,216.0,235.0,169.0,186.0,2
3640,0400117,793971.4952,2.0,42.0,1.029928,1000000.0,2066.0,39.0,44.0,40.0,...,44.0,71.0,67.0,71.0,41.0,13.0,66.0,39.0,41.0,2
3643,0400115,9578.3304,1.0,284.0,1.029928,1000000.0,292.0,480.0,295.0,93.0,...,256.0,244.0,432.0,84.0,307.0,275.0,85.0,284.0,410.0,1


In [25]:
dff['hou_cost'] = dff.SMOCP
dff['aff_cost'] = np.where(dff.HINCP>=1,(dff.HINCP*0.333)/12,0)

In [26]:
#who is the unit affordable to based on the rent
dff['unit_aff'] = np.where(dff.HHSz=='1',pd.cut(dff['hou_cost'],bins=unit_afford['1'],labels=inc_lbls),\
                  np.where(dff.HHSz=='2',pd.cut(dff['hou_cost'],bins=unit_afford['2'],labels=inc_lbls),\
                  np.where(dff.HHSz=='3',pd.cut(dff['hou_cost'],bins=unit_afford['3'],labels=inc_lbls),\
                  np.where(dff.HHSz=='4',pd.cut(dff['hou_cost'],bins=unit_afford['4'],labels=inc_lbls),\
                  np.where(dff.HHSz=='5',pd.cut(dff['hou_cost'],bins=unit_afford['5'],labels=inc_lbls),\
                  np.where(dff.HHSz=='6',pd.cut(dff['hou_cost'],bins=unit_afford['6'],labels=inc_lbls),\
                  np.where(dff.HHSz=='7',pd.cut(dff['hou_cost'],bins=unit_afford['7'],labels=inc_lbls),\
                  np.where(dff.HHSz=='8',pd.cut(dff['hou_cost'],bins=unit_afford['8'],labels=inc_lbls),''))))))))

In [27]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,SMOCP,WGTP1,WGTP2,WGTP3,...,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,HHSz,hou_cost,aff_cost,unit_aff
3602,0400112,76214.6720,2.0,55.0,1.029928,1000000.0,3176.0,56.0,96.0,91.0,...,54.0,16.0,93.0,51.0,56.0,94.0,2,3176.0,2114.957148,o100_ami
3607,0400116,138010.3520,6.0,53.0,1.029928,1000000.0,1628.0,15.0,16.0,62.0,...,60.0,57.0,18.0,90.0,58.0,54.0,6,1628.0,3829.787268,50_80_ami
3618,0400122,32339.7392,2.0,109.0,1.029928,1000000.0,717.0,96.0,152.0,114.0,...,128.0,108.0,216.0,235.0,169.0,186.0,2,717.0,897.427763,30_50_ami


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [28]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [29]:
drop_cols = ['HINCP','NP','ADJINC','ADJHSG','SMOCP','HHSz','hou_cost']

In [30]:
table = dff.copy().drop(columns=drop_cols)

In [31]:
table = table.groupby(['GEO_ID','unit_aff']).sum().reset_index()

In [32]:
table.WGTP.sum()

412215.0

In [33]:
table.head()

,GEO_ID,unit_aff,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,WGTP7,...,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80,aff_cost
0,0400112,30_50_ami,8300.0,8424.0,8119.0,8565.0,8251.0,7875.0,7809.0,7983.0,...,7467.0,7549.0,9395.0,8669.0,7644.0,8532.0,8238.0,7419.0,8162.0,3.689143e+05
1,0400112,50_80_ami,6672.0,6934.0,6306.0,7423.0,7046.0,6328.0,6400.0,7613.0,...,7230.0,6864.0,6273.0,6915.0,6535.0,6708.0,6688.0,6761.0,6614.0,3.360704e+05
2,0400112,80_100_ami,4385.0,4457.0,4687.0,4150.0,4191.0,4057.0,4336.0,3979.0,...,4245.0,4462.0,4723.0,4671.0,4539.0,3914.0,3814.0,4715.0,3926.0,2.039939e+05
3,0400112,o100_ami,22577.0,22345.0,22249.0,23189.0,22616.0,21880.0,22763.0,22623.0,...,22785.0,22631.0,21931.0,22582.0,22846.0,23718.0,21907.0,22271.0,22248.0,1.561353e+06
4,0400112,u30_ami,4113.0,3300.0,4015.0,4137.0,3885.0,4032.0,3987.0,3916.0,...,3857.0,4405.0,4549.0,4385.0,3738.0,3988.0,4043.0,3977.0,4683.0,1.928869e+05


In [34]:
table_2 = table.copy().groupby(['GEO_ID','unit_aff']).sum().reset_index()
table_2 = make_est(table_2)
table_2 = table_2.drop(columns=repwts)

In [35]:
table_2 = pd.pivot_table(table_2,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['unit_aff'],aggfunc=np.sum).reset_index()

In [36]:
table_2

GEO_ID        hh                                            hh_CV  \
unit_aff          30_50_ami 50_80_ami 80_100_ami o100_ami u30_ami  30_50_ami   
0         0400112    8300.0    6672.0     4385.0  22577.0  4113.0   4.536914   
1         0400113    5452.0    4768.0     3832.0  12839.0  3409.0   5.328700   
2         0400114    3874.0    7078.0     3653.0   7597.0  5940.0   7.988610   
3         0400115    3420.0    7485.0     2969.0   4159.0  6370.0   8.048349   
4         0400116    3930.0    4695.0     3075.0   6842.0  3123.0   7.140059   
5         0400117    4131.0    4333.0     2137.0   9541.0  3934.0   7.904726   
6         0400118    2725.0    2463.0     1289.0   4993.0  3919.0  10.212582   
7         0400119    2842.0    7136.0     2749.0   3553.0  6878.0   8.199005   
8         0400120    3122.0    4886.0     3827.0  12324.0  6159.0   6.290526   
9         0400121    3205.0   11748.0     5127.0   4704.0  7709.0   8.919004   
10        0400122    2489.0    5403.0     1076.0    595.0  7168.0   9.035255   
11        0400123    4162.0    7973.0     1711.0   1215.0  5108.0   8.896650   
12        0400125    3832.0    9410.0     2502.0   3114.0  6690.0   8.234372   
13        0400128    4597.0    7312.0     3886.0   8981.0  5215.0   6.301241   
14        0400129    5023.0    6525.0     5299.0  20521.0  6417.0   5.691039   

                                                         hh_MOE               \
unit_aff 50_80_ami 80_100_ami   o100_ami   u30_ami    30_50_ami    50_80_ami   
0         6.569952   5.812182   2.466847  7.358759  1018.991263  1186.178463   
1         5.866235   7.718014   3.722032  5.617516   786.156368   756.880839   
2         5.376413   9.494213   5.298143  6.457778   837.457205  1029.757518   
3         6.323645   9.215417   7.377931  6.418357   744.842985  1280.828786   
4         6.242014   9.487844   4.212192  7.613047   759.322316   793.034650   
5         8.403640  12.980968   4.195508  7.584625   883.636904   985.344152   
6         8.219859  10.859065   7.148850  7.333626   753.067408   547.848660   
7         5.269996  10.428567   9.896058  5.274727   630.546385  1017.646381   
8         6.823166   7.668854   3.676837  6.037335   531.436848   902.134333   
9         4.860723   7.965772   7.792506  4.909254   773.528301  1545.242507   
10        7.082196  15.782219  22.441062  6.705922   608.551181  1035.463312   
11        5.984340  13.867948  12.883922  6.419095  1001.983091  1291.129576   
12        5.441887   9.576937  11.004553  5.316040   853.862164  1385.705615   
13        6.442749   8.572980   5.209915  6.097807   783.848973  1274.791633   
14        6.278786   7.620861   3.587198  5.294554   773.546754  1108.633498   

                                                 
unit_aff   80_100_ami     o100_ami      u30_ami  
0          689.668825  1507.093618   819.021158  
1          800.318493  1293.132711   518.206855  
2          938.513336  1089.174829  1038.009534  
3          740.383964   830.338742  1106.356518  
4          789.486041   779.871495   643.372255  
5          750.660202  1083.203990   807.421485  
6          378.771386   965.894182   777.724616  
7          775.766809   951.457178   981.734326  
8          794.183274  1226.190216  1006.206847  
9         1105.154505   991.919075  1024.106968  
10         459.528181   361.320140  1300.733471  
11         642.087194   423.600197   887.271394  
12         648.404091   927.305483   962.379094  
13         901.501387  1266.155816   860.517723  
14        1092.772522  1991.983049   919.376100

In [37]:
table_2.to_excel('output/pums_own_all.xlsx')